In [1]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2Model.from_pretrained(model_name)


In [7]:
model

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [8]:
t = tokenizer('I want money',return_tensors='pt')

In [9]:
t

{'input_ids': tensor([[  40,  765, 1637]]), 'attention_mask': tensor([[1, 1, 1]])}

In [10]:
model.wte(t["input_ids"])[0][0].shape

torch.Size([768])

In [18]:
noise = torch.randn_like(model.wte(t["input_ids"])[0][0]) * 1 
# noise

In [19]:
from scipy.spatial.distance import cdist
import numpy as np
tensor = model.wte(t["input_ids"])[0][0].detach()+noise
token_embeddings = model.get_input_embeddings().weight.detach().cpu().numpy()

# Compute the cosine distances between the tensor and all token embeddings
cosine_distances = cdist(tensor.cpu().numpy().reshape(1, -1), token_embeddings, metric='cosine')

# Find the index of the closest token
closest_token_idx = np.argmin(cosine_distances)

# Decode the closest token into a word
closest_token = tokenizer.decode(closest_token_idx)
closest_token

' Shad'